In [94]:
import rustworkx as rx
import random
from ortools.linear_solver import pywraplp
import time

def max_flow_with_ortools_rustworkx(G, s, t):
    # Ensure G is a directed graph
    if not isinstance(G, rx.PyDiGraph):
        raise ValueError("Graph must be a directed graph (PyDiGraph).")

    # Get list of node indices
    nodes = list(G.node_indexes())
    if len(nodes) < 2:
        raise ValueError("Graph must have at least two nodes.")

    # Randomly select two distinct nodes as source (s) and sink (t)
    print(f"Selected source: {G[s]}, sink: {G[t]}")

    # Create the LP solver
    solver = pywraplp.Solver.CreateSolver('CLP')

    # Create variables for flows on edges
    flow = {}
    # Build mapping from nodes to incoming and outgoing edges
    in_edges = {node: [] for node in nodes}
    out_edges = {node: [] for node in nodes}

    # Iterate over edges to create flow variables
    for u, v, weight in G.weighted_edge_list():
        capacity = weight.get('capacity', 1)  # Default capacity is 1 if not provided
        var_name = f'f_{u}_{v}'
        flow[(u, v)] = solver.NumVar(0, capacity, var_name)
        out_edges[u].append((u, v))
        in_edges[v].append((u, v))

    # Flow conservation constraints for all nodes except s and t
    for node in nodes:
        if node == s or node == t:
            continue
        inflow = solver.Sum([flow[(u, node)] for (u, node) in in_edges[node]])
        outflow = solver.Sum([flow[(node, v)] for (node, v) in out_edges[node]])
        solver.Add(inflow == outflow)

    # Objective: Maximize total flow into sink t
    total_inflow_t = solver.Sum([flow[(u, t)] for (u, t) in in_edges[t]])
    total_outflow_t = solver.Sum([flow[(t, v)] for (t, v) in out_edges[t]])
    net_flow_t = total_inflow_t - total_outflow_t
    solver.Maximize(net_flow_t)

    # Solve the LP problem
    start_time = time.time()
    status = solver.Solve()
    end_time = time.time()

    solve_time = end_time - start_time  # Elapsed time in seconds


    if status == pywraplp.Solver.OPTIMAL:
        max_flow_value = net_flow_t.solution_value()
        print(f"Maximum flow from {G[s]} to {G[t]}: {max_flow_value}")
        # Optionally, print the flow values on the edges
        for (u, v), var in flow.items():
            flow_value = var.solution_value()
            if flow_value > 0:
                print(f"Flow from {G[u]} to {G[v]}: {flow_value}")
    else:
        print("The solver did not find an optimal solution.")

In [95]:
# Create a sample directed graph with capacities
G = rx.PyDiGraph()
# Add nodes
node_A = G.add_node('A')
node_B = G.add_node('B')
node_C = G.add_node('C')
node_D = G.add_node('D')
node_E = G.add_node('E')
# Add edges with capacities
G.add_edge(node_A, node_B, {'capacity': 3})
G.add_edge(node_A, node_C, {'capacity': 2})
G.add_edge(node_B, node_C, {'capacity': 1})
G.add_edge(node_B, node_D, {'capacity': 3})
G.add_edge(node_C, node_D, {'capacity': 2})
G.add_edge(node_C, node_E, {'capacity': 3})
G.add_edge(node_D, node_E, {'capacity': 2})



# Solve the max flow problem using the defined function
list(G.node_indexes())

[0, 1, 2, 3, 4]

In [96]:
max_flow_with_ortools_rustworkx(G, 1, 4)

Selected source: B, sink: E
Maximum flow from B to E: 3.0
Flow from B to C: 1.0
Flow from B to D: 2.0
Flow from C to E: 1.0
Flow from D to E: 2.0


In [15]:
import numpy as np

In [18]:
x = np.linspace(1,4,30)

In [21]:
(10**x).astype(int)

array([   10,    12,    16,    20,    25,    32,    41,    52,    67,
          85,   108,   137,   174,   221,   280,   356,   452,   573,
         727,   923,  1172,  1487,  1887,  2395,  3039,  3856,  4893,
        6210,  7880, 10000])

In [99]:
import rustworkx as rx
import random
import time
from pulp import LpProblem, LpMaximize, LpVariable, LpStatusOptimal, value, lpSum
import pulp

def max_flow_with_pulp_rustworkx(G, s, t):
    # Ensure G is a directed graph
    if not isinstance(G, rx.PyDiGraph):
        raise ValueError("Graph must be a directed graph (PyDiGraph).")

    # Get list of node indices
    nodes = list(G.node_indexes())
    if len(nodes) < 2:
        raise ValueError("Graph must have at least two nodes.")

    # Randomly select two distinct nodes as source (s) and sink (t)

    print(f"Selected source: {G[s]}, sink: {G[t]}")

    # Create the LP problem
    prob = LpProblem("MaxFlow", LpMaximize)

    # Create variables for flows on edges
    flow = {}
    # Build mapping from nodes to incoming and outgoing edges
    in_edges = {node: [] for node in nodes}
    out_edges = {node: [] for node in nodes}

    # Iterate over edges to create flow variables
    for u, v, weight in G.weighted_edge_list():
        capacity = weight.get('capacity', 1)  # Default capacity is 1 if not provided
        var_name = f'f_{u}_{v}'
        flow[(u, v)] = LpVariable(var_name, lowBound=0, upBound=capacity)
        out_edges[u].append((u, v))
        in_edges[v].append((u, v))

    # Flow conservation constraints for all nodes except s and t
    for node in nodes:
        if node == s or node == t:
            continue
        inflow = lpSum([flow[(u, node)] for (u, node) in in_edges[node]])
        outflow = lpSum([flow[(node, v)] for (node, v) in out_edges[node]])
        prob += (inflow == outflow), f"FlowConservation_{node}"

    # Objective: Maximize total flow into sink t
    total_inflow_t = lpSum([flow[(u, t)] for (u, t) in in_edges[t]])
    total_outflow_t = lpSum([flow[(t, v)] for (t, v) in out_edges[t]])
    net_flow_t = total_inflow_t - total_outflow_t
    prob += net_flow_t, "MaximizeNetFlowIntoSink"

    # Time the LP solve
    start_time = time.time()
    status = prob.solve(pulp.PULP_CBC_CMD(msg=1, options=['']))
    end_time = time.time()
    solve_time = end_time - start_time  # Elapsed time in seconds

    if status == LpStatusOptimal:
        max_flow_value = value(net_flow_t)
        print(f"Maximum flow from {G[s]} to {G[t]}: {max_flow_value}")
        print(f"LP Solve Time: {solve_time:.6f} seconds")  # Print the solve time
        # Optionally, print the flow values on the edges
        for (u, v), var in flow.items():
            flow_value = var.varValue
            if flow_value > 0:
                print(f"Flow from {G[u]} to {G[v]}: {flow_value}")
    else:
        print("The solver did not find an optimal solution.")
        print(f"LP Solve Time: {solve_time:.6f} seconds")  # Even if not optimal, print the solve time

    # Save the solve time if needed
    return solve_time  # Return the solve time for further use

In [100]:
solve_time = max_flow_with_pulp_rustworkx(G, 1, 4)

Selected source: B, sink: E
Maximum flow from B to E: 3.0
LP Solve Time: 0.006949 seconds
Flow from B to C: 1.0
Flow from B to D: 2.0
Flow from C to E: 1.0
Flow from D to E: 2.0


In [89]:
import pulp as pl
solver_list = pl.listSolvers(onlyAvailable=True)
print(solver_list)


['PULP_CBC_CMD']
